In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import datetime
import pandas as pd
import torch
import torch.nn as nn
sys.path.append('../')
from lr_scheduler import CyclicLR
from training_utils import training_loop, test_loop
from model import RNNLM
from data_utils import (IndexVectorizer, 
                        TextDataset, 
                        SpacyTokenizer,
                        LMDataLoader)
import pickle

In [3]:
####################################################
# Config
####################################################

## Input / output
data_dir = '../../../..'

## Tokenization
TOKENIZE = SpacyTokenizer().tokenize

## Vectorization
MIN_WORD_FREQ = 2
MAX_VOCAB_SIZE = 20000
STAT_END_TOK = True

## Model Architecture
hidden_dim = 300
embedding_dim = 300
dropout = 0.5
lstm_layers = 3
lstm_bidirection = True

## Training Language Model
batch_size = 80
learning_rate = 1e-3
num_epochs = 100
display_epoch_freq = 10
target_seq_len = 100
max_seq_len = 130
min_seq_len = 20

In [4]:
# GPU setup
use_gpu = torch.cuda.is_available()
device_num = 0
device = torch.device(f"cuda:{device_num}" if use_gpu else "cpu")
device

device(type='cuda', index=0)

In [5]:
# IO setup
today = datetime.datetime.now().strftime('%Y-%m-%d')
model_cache_dir = os.path.join(data_dir, 'models')
data_cache = os.path.join(model_cache_dir, 'data_cache.pkl')
vectorizer_cache = os.path.join(model_cache_dir, 'lm_vectorizer.pkl')
os.makedirs(model_cache_dir, exist_ok=True)
model_file_lm = os.path.join(model_cache_dir, f'LM__{today}.json')
model_file_class = os.path.join(model_cache_dir, f'CLASS__{today}.json')

train_file = os.path.join(data_dir, 'train.csv')
valid_file = os.path.join(data_dir, 'valid.csv')

In [6]:
RE_VECTORIZE = False
if RE_VECTORIZE or not os.path.isfile(data_cache):
    train = pd.read_csv(train_file)
    valid = pd.read_csv(valid_file)
    vectorizer = IndexVectorizer(max_words = MAX_VOCAB_SIZE, 
                             min_frequency=MIN_WORD_FREQ,
                             start_end_tokens=STAT_END_TOK, 
                             tokenize=TOKENIZE)
    train_ds = TextDataset(data=train, vectorizer=vectorizer, text_col='text')
    valid_ds = TextDataset(data=valid, vectorizer=vectorizer, text_col='text')
    pickle.dump([train_ds, valid_ds], open(data_cache, 'wb'))
    pickle.dump(vectorizer, open(vectorizer_cache, 'wb'))
else:
    train_ds, valid_ds = pickle.load(open(data_cache, 'rb'))
    vectorizer = pickle.load(open(vectorizer_cache, 'rb'))

In [7]:
print(f'Train size: {len(train_ds)}\nvalid size: {len(valid_ds)}')
print(f"Vocab size: {len(vectorizer.vocabulary)}")

Train size: 5000
valid size: 1000
Vocab size: 11665


In [8]:
train_dl = LMDataLoader(dataset=train_ds, 
                        target_seq_len=target_seq_len, 
                        shuffle=True, 
                        max_seq_len=max_seq_len, 
                        min_seq_len=min_seq_len, 
                        p_half_seq_len=0.05,
                        batch_size=batch_size)
valid_dl = LMDataLoader(dataset=valid_ds,
                        target_seq_len=target_seq_len, 
                        shuffle=True, 
                        max_seq_len=max_seq_len, 
                        min_seq_len=min_seq_len, 
                        p_half_seq_len=0.05,
                        batch_size=batch_size) 

In [9]:
if use_gpu: torch.cuda.manual_seed(303)
else: torch.manual_seed(303)

In [16]:
# set up Files to save stuff in
runtime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
model_file_lm = model_file_lm
    
TIE_WEIGHTS = True 
NUM_LAYERS = 3
BIDRIECTIONAL = True

# Build and initialize the model
lm = RNNLM(device, vectorizer.vocabulary_size, embedding_dim, hidden_dim, batch_size, 
           dropout = dropout, 
           tie_weights = TIE_WEIGHTS, 
           num_layers = NUM_LAYERS, 
           bidirectional = BIDRIECTIONAL, 
           word2idx = vectorizer.word2idx,
           log_softmax = False)

In [17]:
if use_gpu:
    lm = lm.to(device)
#lm.init_weights()

In [ ]:
# Loss and Optimizer
loss = nn.CrossEntropyLoss()

# Extract pointers to the parameters of the lstms
param_list = [{'params': rnn.parameters(), 'lr': 1e-3} for rnn in lm.rnns]

# If weights are tied between encoder and decoder, we can only optimize 
# parameters in one of those two layers
if not TIE_WEIGHTS:
    param_list.extend([
            {'params': lm.encoder.parameters(), 'lr':1e-3},
            {'params': lm.decoder.parameters(), 'lr':1e-3},
        ])
else:
    param_list.extend([
        {'params': lm.encoder.parameters(), 'lr':1e-3},
    ])

optimizer = torch.optim.Adam(param_list, lr=0.01)

scheduler = CyclicLR(optimizer,  max_lrs=[0.1, 0.01, 0.01, 0.01, 0.1], 
                     mode='ulmfit', ratio=1.5, cut_frac=0.4, 
                     n_epochs=num_epochs, batchsize=50000/1171, 
                     verbose=False, epoch_length=50000)
history = training_loop(batch_size, num_epochs, 1, 
                        lm, loss, optimizer, scheduler, device, 
                        train_dl, valid_dl, 
                        best_model_path=model_file_lm)

Epoch: 0000; Loss: 6.7565; Val-Loss 6.2569; Perplexity 859.5862; Val-Perplexity 521.5804
Sample: this is good most hour . take started bite a ever better know . everywhere , a because went local it dish a und have a purchase could upon walked he they next <END> worth 
 live beer heard desired were she 

 ! fried fresh great change the come any appetizer combination
Epoch: 0001; Loss: 6.1147; Val-Loss 5.3322; Perplexity 452.4729; Val-Perplexity 206.8927
Sample: this is good usually - even on decent and a my great the maybe for a sure came and lack to great ! wait time although that i 've lunch needed i little is tenders and or work will edges sauce by my done and salon should reason room a very what not
Epoch: 0002; Loss: 5.4599; Val-Loss 4.6182; Perplexity 235.0816; Val-Perplexity 101.3119
Sample: this is good there $ refreshing find 

 everything from been ! from <END> while and in definitely when 
 way support recommended you my our be cool arrived stays step my seems ) one intereste

In [30]:
torch.cuda.empty_cache()